In [1]:
%load_ext autoreload
%autoreload 2
    
import os, sys
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from emimage.models import EMImage
from mims.models import MIMSImage, MIMSImageSet
from PIL import Image
from scipy.ndimage import rotate
import pprint
import sims

In [3]:

em_image = EMImage.objects.all().order_by('-created_at').first()
mims_images = list(em_image.mimsimageset_set.first().mims_images.all())

coordinates = []
for mims in mims_images:
    im = sims.SIMS(mims.file.path)
    x = im.header['sample x']
    y = im.header['sample y']
    coordinates.append((mims, (x, y)))

centroid_x = sum(coord[1][0] for coord in coordinates) / len(coordinates)
centroid_y = sum(coord[1][1] for coord in coordinates) / len(coordinates)

# Find the mims object with the coordinate closest to the centroid
def distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

centroid = (centroid_x, centroid_y)
closest_mims, closest_coordinate = min(coordinates, key=lambda coord: distance(coord[1], centroid))

print(f"Centroid: ({centroid_x}, {centroid_y})")
print(f"Closest coordinate: {closest_coordinate}")
print(f"Original mims object: {closest_mims}")

Centroid: (-9944.416666666666, 16640.416666666668)
Closest coordinate: (-9927, 16660)
Original mims object: mims_image_sets/221a6372-a491-4e8a-aac0-fe100634fefd/mims_images/1036_position_7.im


In [4]:
# Function to get autocontrast image path
def get_autocontrast_image_path(mims_image, species, media_root):
    mims_image_set = mims_image.image_set
    isotope_image_dir = os.path.join(
        media_root,
        "mims_images",
        str(mims_image_set.id),
        str(mims_image.id),
        "isotopes",
    )
    autocontrast_path = os.path.join(
        isotope_image_dir, f"{species}_autocontrast.png"
    )
    return autocontrast_path

# Function to load images and their positions
def load_images_and_positions(em_image, species, media_root):
    mims_image_set = em_image.mimsimageset_set.first()
    mims_images = list(mims_image_set.mims_images.all())
    
    images = []
    positions = []
    
    for mims in mims_images:
        im = sims.SIMS(mims.file.path)
        with open("./sims_header.txt", "w") as f:
            pprint.pprint(im.header, f)
        x, y = im.header['sample x'], im.header['sample y']
        x,y = y,x
        
        autocontrast_path = get_autocontrast_image_path(mims, species, media_root)
        if os.path.exists(autocontrast_path):
            image = Image.open(autocontrast_path)
            images.append(np.array(image))
            positions.append((x, y))
    
    return images, positions

# Function to create and display the concatenated image
def create_and_display_concatenated_image(em_image, species, media_root, rotation_angle=None, flip=False):
    images, positions = load_images_and_positions(em_image, species, media_root)
    mims_meta = sims.SIMS(em_image.mimsimageset_set.first().mims_images.first().file.path).header["Image"]
    mims_pixel_size = mims_meta["raster"] / mims_meta["width"]
    
    if not images:
        print("No images found for the given species.")
        return
    
    # Assuming all images are the same size
    image_height, image_width = images[0].shape

    # Determine the extent of the concatenated image
    min_x = min(pos[0] for pos in positions)
    min_y = min(pos[1] for pos in positions)
    max_x = max(pos[0] for pos in positions)
    max_y = max(pos[1] for pos in positions)
    
    canvas_width = int((max_x - min_x)  * 1000/mims_pixel_size + image_width)
    canvas_height = int((max_y - min_y)  * 1000/mims_pixel_size + image_height)
    
    # Create a blank canvas
    canvas = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    
    # Place each image on the canvas
    for img, (x, y) in zip(images, positions):
        x_offset = int((x - min_x) * 1000 / mims_pixel_size)
        y_offset = int((max_y - y) * 1000 / mims_pixel_size)
        canvas[y_offset:y_offset+image_height, x_offset:x_offset+image_width] = img


    if rotation_angle:
        canvas = rotate(canvas, -rotation_angle, reshape=True, mode='constant', cval=0)

    if flip:
        canvas = np.fliplr(canvas)
    
    # Display the concatenated image
    plt.figure(figsize=(20,20))
    plt.imshow(canvas, cmap='gray')
    plt.title(f"Concatenated MIMS Image for {species}")
    plt.show()

  # Replace with the actual media root path
species = '32S'  # Replace with the actual isotope species
create_and_display_concatenated_image(em_image, species, media_root)#, rotation_angle=262, flip=True)


No images found for the given species.


In [5]:
def load_images_and_positions(em_image, species, media_root):
    mims_image_set = em_image.mimsimageset_set.first()
    mims_images = list(mims_image_set.mims_images.all())
    
    images = []
    positions = []
    max_x = -999999
    max_y = -999999
    for mims in mims_images:
        im = sims.SIMS(mims.file.path)
        x, y = im.header['sample x'], im.header['sample y']
        max_x = max(x, max_x)
        max_y = max(y, max_y)
    for mims in mims_images:
        im = sims.SIMS(mims.file.path)
        x, y, z = im.header['sample x'], im.header['sample y'], im.header['sample z']
        positions.append((max_x-x, max_y-y))
    return positions
      
pos = load_images_and_positions(em_image, species, media_root)
print(pos)

[(157, 243), (157, 193), (141, 138), (81, 118), (36, 194), (0, 16), (47, 50), (109, 77), (127, 19), (208, 0), (245, 30), (209, 81)]


In [8]:
from mims.tasks import preprocess_mims_image_set
from mims.models import MIMSImageSet
import pyvips
from mims.services.concat_utils import get_concatenated_image

mims_image_set = MIMSImageSet.objects.get(id="f6f4163c-1fe3-4c25-901f-2afef795f6cd")
print(mims_image_set.em_image.delete())

for isotope in mims_image_set.mims_images.first().isotopes.all():
    output_dir = os.path.join(
        settings.MEDIA_ROOT,
        "mims_images",
        str(mims_image_set.id),
        "composites",
        "isotopes",
    )
    os.makedirs(output_dir, exist_ok=True)
    img = get_concatenated_image(mims_image_set.em_image, isotope.name)
    img = pyvips.Image.new_from_array(img)

    # Save the DZI file directly to the target directory
    img.dzsave(
        os.path.join(output_dir, isotope.name + ".dzi"),
        layout=pyvips.enums.ForeignDzLayout.IIIF3,
    )
print("done wtih ", mims_image_set_id)

None


AttributeError: 'NoneType' object has no attribute 'isotopes'

In [ ]:
#from mims.models import MIMSAlignment
from mims.services.image_utils import (
    manipulate_image,
    update_top_locations,
    do_sliding_search,
    correct_inner_zeros,
)
from scipy import ndimage as ndi
from skimage.util import random_noise
from skimage import feature

em_image = EMImage.objects.all().order_by('-created_at').first()
sem_image_path = em_image.file.path
sem_image = pyvips.Image.new_from_file(sem_image_path).numpy()
# edges = feature.canny(sem_image, sigma=3)

#plt.figure(figsize=(20,20))
#plt.imshow(edges, cmap='gray')
#plt.show()

In [ ]:
def find_central_roi(em_image, media_root):
    mims_images = list(em_image.mimsimageset_set.first().mims_images.all())

    coordinates = []
    for mims in mims_images:
        im = sims.SIMS(mims.file.path)
        x = im.header['sample x']
        y = im.header['sample y']
        coordinates.append((mims, (x, y)))

    centroid_x = sum(coord[1][0] for coord in coordinates) / len(coordinates)
    centroid_y = sum(coord[1][1] for coord in coordinates) / len(coordinates)

    # Find the mims object with the coordinate closest to the centroid
    def distance(coord1, coord2):
        return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

    centroid = (centroid_x, centroid_y)
    closest_mims, closest_coordinate = min(coordinates, key=lambda coord: distance(coord[1], centroid))

    return closest_mims, closest_coordinate

from pathlib import Path
mims_obj = find_central_roi(em_image, media_root)[0]
print("Central MIMS obj is ",mims_obj)
# Function to get autocontrast image path
mims_path = get_autocontrast_image_path(mims_obj, "32S", media_root)
try:
    mims = pyvips.Image.new_from_file(mims_path).numpy()
except Exception as e:
    print("oh no", e)

Central MIMS obj is  mims_images/963_ROI_7.im


In [ ]:
from pystackreg import StackReg
from skimage import io

sem_image_inv = np.invert(sem_image)
THRESHOLD_8BIT_BUFFER = 10

sem_threshold = np.percentile(sem_image_inv, 90) - THRESHOLD_8BIT_BUFFER
sem_image_inv_thr = np.where(sem_image_inv > sem_threshold, 255, 0).astype(np.uint8)
mims_threshold = np.percentile(mims, 90) - THRESHOLD_8BIT_BUFFER
mims_thr = np.where(mims > mims_threshold, 255, 0).astype(np.uint8)

#sr = StackReg(StackReg.SCALED_ROTATION)

In [ ]:
# Pad the mims_thr with 0s to be the same shape as sem_image_inv_thr
padding = [(0, sem_image_inv_thr.shape[dim] - mims_thr.shape[dim]) for dim in range(len(sem_image_inv_thr.shape))]
mims_padded = np.pad(mims_thr, padding, mode='constant', constant_values=0)
#out = sr.register_transform(sem_image_inv_thr, mims_padded)

In [ ]:
import os
import numpy as np
import pyvips
import sims
from PIL import Image
import matplotlib.pyplot as plt
from django.conf import settings
from mims.models import MIMSAlignment
from mims.services.image_utils import (
    manipulate_image,
    update_top_locations,
    do_sliding_search,
    correct_inner_zeros,
)

# Function to get autocontrast image path
def get_autocontrast_image_path(mims_image, species, media_root):
    mims_image_set = mims_image.image_set
    isotope_image_dir = os.path.join(
        media_root,
        "mims_images",
        str(mims_image_set.id),
        str(mims_image.id),
        "isotopes",
    )
    autocontrast_path = os.path.join(
        isotope_image_dir, f"{species}_autocontrast.png"
    )
    return autocontrast_path

# Function to load images and their positions
def load_images_and_positions(em_image, species, media_root):
    mims_image_set = em_image.mimsimageset_set.first()
    mims_images = list(mims_image_set.mims_images.all())
    
    images = []
    positions = []
    
    for mims in mims_images:
        im = sims.SIMS(mims.file.path)
        x, y = im.header['sample x'], im.header['sample y']
        x, y = y, x
        
        autocontrast_path = get_autocontrast_image_path(mims, species, media_root)
        if os.path.exists(autocontrast_path):
            image = Image.open(autocontrast_path)
            images.append(np.array(image))
            positions.append((x, y))
    
    return images, positions

# Function to find the central ROI
def find_central_roi(em_image, media_root):
    """Identify the central region of interest (ROI)."""
    mims_images = list(em_image.mimsimageset_set.first().mims_images.all())
    coordinates = [(mims, (sims.SIMS(mims.file.path).header['sample x'], sims.SIMS(mims.file.path).header['sample y'])) for mims in mims_images]

    centroid_x = np.mean([coord[1][0] for coord in coordinates])
    centroid_y = np.mean([coord[1][1] for coord in coordinates])
    
    centroid = (centroid_x, centroid_y)
    closest_mims, _ = min(coordinates, key=lambda coord: np.hypot(coord[1][0] - centroid[0], coord[1][1] - centroid[1]))
    
    return closest_mims

# Function to perform threshold matching
def threshold_match2(im1_full, im2_full, THRESHOLD_8BIT_BUFFER=10, angle=None, flip_hor=None):
    def binarize_image(image, threshold):
        return np.where(image > threshold, 255, 0).astype(np.uint8)
    valid_im1 = im1_full > 0
    valid_im2 = im2_full > 0
    im1_threshold = np.percentile(im1_full, 90) - THRESHOLD_8BIT_BUFFER
    im2_threshold = np.percentile(im2_full, 90) - THRESHOLD_8BIT_BUFFER
    im1 = binarize_image(im1_full, im1_threshold)
    im2 = binarize_image(im2_full, im2_threshold)

    best_overall_locations = []

    if angle is not None and flip_hor is not None:
        im2_manipulated = manipulate_image(im2, angle, flip_hor)
        best_overall_locations = [
            (i, (angle, flip_hor))
            for i in do_sliding_search(
                im1, im2_manipulated, valid_im1, valid_im2, angle, flip_hor
            )
        ]
    else:
        for test_angle in range(0, 360, 15):
            for test_flip_hor in [True, False]:
                im2_manipulated = manipulate_image(im2, test_angle, test_flip_hor)
                valid_im2 = im2_manipulated > 0
                im2_manipulated = np.where(
                    im2_manipulated > im2_threshold, 255, 0
                ).astype(np.uint8)
                best_for_iteration = do_sliding_search(
                    im1,
                    im2_manipulated,
                    valid_im1,
                    valid_im2,
                    test_angle,
                    test_flip_hor,
                )
                for top_for_iteration in best_for_iteration:
                    best_overall_locations = update_top_locations(
                        best_overall_locations,
                        top_for_iteration[0],
                        top_for_iteration[1],
                        top_for_iteration[2],
                        top_for_iteration[3],
                        top_for_iteration[4],
                    )

    return best_overall_locations

# Function to calculate IoU score
def calculate_iou(im1, im2):
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

# Main function
def align_mims_images_with_sem(em_image, species, media_root):
    closest_mims, closest_coordinate = find_central_roi(em_image, media_root)
    images, positions = load_images_and_positions(em_image, species, media_root)
    
    central_image_path = get_autocontrast_image_path(closest_mims, species, media_root)
    central_image = np.array(Image.open(central_image_path))
    
    # Load and prepare the SEM image
    sem_image_path = em_image.file.path
    sem_image = pyvips.Image.new_from_file(sem_image_path).numpy()
    # Invert it to match the 32S image
    sem_image = np.invert(sem_image)
    sem_image = correct_inner_zeros(sem_image).astype(np.uint8)
    
    # Set inner 0s to 1s so we can expand with more 0s
    sem_image = correct_inner_zeros(sem_image).astype(np.uint8)
    # Expand the image to include padding for the sliding check
    EM_PADDING = int(central_image.shape[0] * 0.6)
    sem_image = (
        pyvips.Image.new_from_array(sem_image)
        .embed(
            EM_PADDING,
            EM_PADDING,
            sem_image.shape[1] + 2 * EM_PADDING,
            sem_image.shape[0] + 2 * EM_PADDING,
            background=0,
        )
        .numpy()
    )
    
    
    best_positions = threshold_match2(sem_image, central_image, 10)
    print("got best positions for the central image:", best_positions)
    top_3_positions = []
    for pos in best_positions:
        print("checking the first one")
        x_offset, y_offset, angle, flip_hor, _ = pos
        iou_scores = []
        
        for img, (x, y) in zip(images, positions):
            print("starting subcheck on the first best_pos")
            if abs(x - x_offset) <= 250 and abs(y - y_offset) <= 250:
                manipulated_img = manipulate_image(img, angle, flip_hor)
                sem_subimage = sem_image[y_offset:y_offset+manipulated_img.shape[0], x_offset:x_offset+manipulated_img.shape[1]]
                iou = calculate_iou(manipulated_img, sem_subimage)
                iou_scores.append(iou)
        
        overall_iou = np.mean(iou_scores)
        top_3_positions.append((x_offset, y_offset, angle, flip_hor, overall_iou))
    
    top_3_positions = sorted(top_3_positions, key=lambda x: x[4], reverse=True)[:3]
    
    # Display the best one
    best_position = top_3_positions[0]
    x_offset, y_offset, angle, flip_hor, _ = best_position
    best_img = manipulate_image(central_image, angle, flip_hor)
    sem_subimage = sem_image[y_offset:y_offset+best_img.shape[0], x_offset:x_offset+best_img.shape[1]]
    
    plt.figure(figsize=(20, 20))
    plt.imshow(sem_subimage, cmap='gray')
    plt.imshow(best_img, cmap='jet', alpha=0.5)
    plt.title(f"Best Match for {species}")
    plt.show()
    
    return top_3_positions

# Example usage
media_root = settings.MEDIA_ROOT  # Replace with the actual media root path
species = '32S'  # Replace with the actual isotope species
em_image = EMImage.objects.all().order_by('-created_at').first()

top_3_positions = align_mims_images_with_sem(em_image, species, media_root)
print(top_3_positions)


KeyboardInterrupt: 